# 1. EDA

In [4]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Configurar estilo de visualización
plt.style.use('seaborn')
sns.set_palette('viridis')

# Crear dataset sintético basado en información pública de YPF (2023-2024)
data = {
    'Trimestre': ['Q1-2023', 'Q2-2023', 'Q3-2023', 'Q4-2023', 'Q1-2024', 'Q2-2024', 'Q3-2024'],
    'Produccion_Petroleo_kbd': [310, 315, 320, 325, 330, 335, 340],  # Miles de barriles/día
    'Produccion_Gas_MMm3d': [37, 38, 39, 40, 41, 42, 43],  # Millones de m³/día
    'Produccion_Shale_Petroleo_kbd': [90, 100, 110, 115, 120, 122, 126],
    'Ventas_Combustibles_MML': [1500, 1550, 1600, 1650, 1620, 1580, 1700],  # Millones de litros
    'Costo_Extraccion_USD_bbl': [12.5, 12.0, 11.8, 11.5, 11.7, 11.9, 11.6],  # USD por barril
    'Ingresos_MUSD': [4000, 4200, 4300, 4500, 4400, 4300, 4600],  # Millones USD
    'EBITDA_MUSD': [1200, 1250, 1300, 1366, 1300, 1204, 1366],
    'CAPEX_MUSD': [1100, 1150, 1200, 1250, 1300, 1200, 1353],
    'Deuda_Neta_MUSD': [6800, 6900, 7000, 7100, 7200, 7457, 7500],
    'Precio_Brent_USD_bbl': [80, 75, 85, 82, 87, 83, 80]
}
df = pd.DataFrame(data)

# Guardar dataset sintético (opcional)
df.to_csv('ypf_data_sintetico.csv', index=False)

# Mostrar primeras filas
print(df.head())